In [ ]:
def generate_random_accusative_scentence(noun_dfs,verb_dfs,sentiment):
#takes in a verb list noun list and then returns a multidimensional array formated like so
#ie i hate romans would be
#[[i,noun,null],[hate,verb,null],[romans,noun,negative]]
#sentiment is attatched to object of the scentence
    random_verb_df = random.choice(verb_dfs)
    random_noun_df = random.choice(noun_dfs)
    #
    i = 0
    num_tries=0
    while True:
        #selects a random row and column in the declension chart
        # will keep on tryin until the num of tries > 20 arbitrary number
        random_verb_row = random.randrange(0,random_verb_df.shape[0])
        random_verb_column = random.randrange(1,random_verb_df.shape[1])
        random_noun_row = random.randrange(0,random_noun_df.shape[0])
        random_noun_column = random.randrange(1,random_noun_df.shape[1])

        verb =strip_accents_and_lowercase(
        random_verb_df.iat[ random_verb_row,random_verb_column])

        noun =strip_accents_and_lowercase(random_noun_df.iat[ random_noun_row,random_noun_column])
        #print(verb)
        if((noun== '' or contains_number(noun)or
        verb=='' or contains_number(verb))
        and num_tries<20 ):
            print("COULDNT FIND WORD NUMBER OF TRIES:", num_tries+1)
            num_tries+=1
            continue
        num_tries = 0
        break
    output = [[verb,"verb",""],[noun,"noun",sentiment]]
    return output

    
def generate_random_nominative_scentence(noun_dfs,verb_dfs,adjective_dfs,sentiment):
    random_noun_df = random.choice(noun_dfs)
    verb=""
    noun=""
    
    #isinstance checks if verb_df is a list
    #mostly to check if the verb is είναι which is the equivlent to "are" as in they are good είναι has not conjugation charts
    if(isinstance(verb_dfs, list)):
        random_verb_df = random.choice(verb_dfs)
    random_adjective_df = random.choice(adjective_dfs)
    #gramatical number means firstperson pluratlity ie i 
    gramatical_number=["1 sg","2 sg","3 sg","1 pl","2 pl","3 pl"]
    #used for selecting singularity and pluratility not used yet right now just takes a random
    #
    i = 0
    num_tries=0
    
    while True:
        
        random_gramatical_number = random.choice(gramatical_number)
        
        
        random_noun_row = random.randrange(0,random_noun_df.shape[0])
        random_noun_column = random.randrange(1,random_noun_df.shape[1])
        #random_noun_column = get_indices(random_noun_dfs[i],random_gramatical_number)
        
        #random_verb_row = get_indices(random_verb_dfs[i],random_gramatical_number)
        
        if(isinstance(verb_dfs, list)):
            random_verb_row = random.randrange(0,random_verb_df.shape[0])
            random_verb_column = random.randrange(1,random_verb_df.shape[1])

        random_adjective_row = random.randrange(0,random_adjective_df.shape[0])
        random_adjective_column = random.randrange(1,random_adjective_df.shape[1])

        noun =strip_accents_and_lowercase(
        random_noun_df.iat[ random_noun_row,random_noun_column])

        if(isinstance(verb_dfs, list)):
            verb =random_verb_df.iat[ random_verb_row,random_verb_column]
        else:
            verb =verb_dfs

        

        adjective =strip_accents_and_lowercase(
        random_adjective_df.iat[ random_adjective_row,random_adjective_column])
        
        if((noun== '' or contains_number(noun)or verb=='' or contains_number(verb)or adjective=='' or contains_number(adjective))and num_tries<20 ):
            print("COULDNT FIND WORD, NUMBER OF TRIES:", num_tries+1)
            num_tries+=1
            continue
        num_tries = 0
        
        
        break
        
    
    output = [[noun,"noun",sentiment],[verb,"verb",""],[adjective,"adjective",""]]
    return output
    
def display_scentnece(scentence):
    for word in scentence:
        display(word[0])

def append_scentence_to_dataset(path,scentence):
    display(path)
    with open(path,"a",encoding="utf-8") as f:
        f.write("[ O -100\n")
        for word in scentence:
            text = word[0].split(" ")
            for i in range(len(text)):
                if(word[2]!=""):
                    if(i==0):
                        f.write(text[i] +" B-ASP "+word[2]+"\n")
                    else:
                        f.write(text[i] +" I-ASP "+word[2]+"\n")
                else:
                    f.write(text[i] +" O "+"-100\n")
        f.write("] O -100\n\n")
    return
def test_create_accusative_scentence(randomize=True,debug=False,sentiment="positive"):

    matchednouns= match_words_in_folder("greek_variations/noun","greek_dictionary/nouns")

    #display_list(get_accustive_nouns(path_to_df(matchednouns)))


    matchedverbs_pos= match_words_in_folder("greek_variations/verb","greek_dictionary/verbs/actionverbs/positive")
    matchedverbs_neg= match_words_in_folder("greek_variations/verb","greek_dictionary/verbs/actionverbs/negative")

    ##display_list(path_to_df(matchedwords))

    #display_list(get_present_singular_verbs(path_to_df(matchedverbs_neg)))

    #if debug:display_list(path_to_df(matchedverbs_pos))
    #verbs_pos=get_present_singular_verbs(path_to_df(matchedverbs_pos))
    verbs_pos = get_items(get_each_df_from_path_list(matchedverbs_pos),"sg","present")
    verbs_neg=get_items(get_each_df_from_path_list(matchedverbs_neg),"sg","present")
    #nouns=get_accustive_nouns(path_to_df(matchednouns))
    #nouns=return_rows_that_contain(['accusative','nominative'],path_to_df(matchednouns))
    nouns=get_items(get_each_df_from_path_list(matchednouns),'accusative')
    #display(nouns)
    if debug:display_list(nouns)
    if debug:display_list(verbs_pos)
    
    if (sentiment == "positive"):
        scentence =generate_random_accusative_scentence(nouns,verbs_pos,sentiment.capitalize())
    elif(sentiment == "negative"):
        scentence =generate_random_accusative_scentence(nouns,verbs_neg,sentiment.capitalize())
    display(scentence)
    append_scentence_to_dataset("greek_output/Intermediate/accusative.atepc",scentence)
    return
    
def test_create_nominative_scentence(randomize=True,debug=False,sentiment="positive"):
    matchednouns= match_words_in_folder("greek_variations/noun","greek_dictionary/nouns")
    matchedverbs= match_words_in_folder("greek_variations/verb","greek_dictionary/verbs/linkingverbs/")
    matched_adj_pos= match_words_in_folder("greek_variations/adjective","greek_dictionary/adjectives/positive")
    matched_adj_neg= match_words_in_folder("greek_variations/adjective","greek_dictionary/adjectives/negative")


    ##display_list(path_to_df(matchedwords))

    #display_list(get_present_singular_verbs(path_to_df(matchedverbs_neg)))

    #if debug:display_list(path_to_df(matchedverbs_pos))
    #verbs_pos=get_present_singular_verbs(path_to_df(matchedverbs_pos))
    nouns=get_items(get_each_df_from_path_list(matchednouns),'nominative')
    #verbs=get_items(path_to_df(matchedverbs),["sg","pl"],"present")
    #display_list(path_to_df(matchedverbs))
    #verbs=get_items(path_to_df(matchedverbs),"infinitive","active")
    verbs= strip_accents_and_lowercase("εἶναι")

    adjectives_pos=get_items(get_each_df_from_path_list(matched_adj_pos),"nominative")
    adjectives_neg=get_items(get_each_df_from_path_list(matched_adj_neg),"nominative")
    #nouns=get_accustive_nouns(path_to_df(matchednouns))
    #nouns=return_rows_that_contain(['accusative','nominative'],path_to_df(matchednouns))

    #display(nouns)
    if debug:display_list(nouns)
    if debug:display_list(verbs)
    if debug:display_list(adjectives_pos)
    
    scentence=[[]]
    
    if (sentiment == "positive"):
        scentence =generate_random_nominative_scentence(nouns,verbs,adjectives_pos,sentiment.capitalize())
    
    elif(sentiment == "negative"):
        scentence =generate_random_nominative_scentence(nouns,verbs,adjectives_neg,sentiment.capitalize())
        
    display(scentence)
    append_scentence_to_dataset("greek_output/Intermediate/nominative.atepc",scentence)
    
    return

